In [159]:
import textract
import pandas as pd
import re
#text = textract.process('lsat_pdfs/270577485-LSAT-PT-72.pdf') -- encrypted
text38 = textract.process('lsat_pdfs/270577397-LSAT-PT-38-pdf.pdf')
text65 = textract.process('lsat_pdfs/270577461-LSAT-PT-65.pdf')
text61 = textract.process('lsat_pdfs/270577450-LSAT-PT-61.pdf')
text60 = textract.process('lsat_pdfs/270577448-LSAT-PT-60.pdf')
text59 = textract.process('lsat_pdfs/270577444-LSAT-PT-59.pdf')
#text = textract.process('lsat_pdfs/270577492-LSAT-PT-74.pdf') -- encrypted?
#text = textract.process('lsat_pdfs/270577487-LSAT-PT-73.pdf') -- encrypted?

In [514]:
def cleanup(text,oldnew={'\n': ' ', '\t': ' ', '\xe2\x80\x99':"'"}):
    for old, new in oldnew.iteritems():
        text = text.replace(old,new)
    return text

def test_number(text,identifier='PrepTest'):
    text = cleanup(text)
    return int(text.split('PrepTest ')[1][:2])

def sections(text,section_begin='answer sheet',section_end='IF YOU'):
    text = cleanup(text)
    section_contents = [section.split(section_end)[0] for section in text.split(section_begin)[1:]]
    section_labels = [x for _,x in sorted(zip(
        [text.find('Logical'),
         text.find('Logical')+text[text.find('Logical')+1:].find('Logical'),
         text.find('Analytical'),
         text.find('Reading')],
        ["Logical Reasoning","Logical Reasoning","Analytical Reasoning","Reading Comprehension"]))]
    section_numbers = [1,2,3,4]
    
    df = pd.DataFrame({'section_content': section_contents,
                      'section_label': section_labels,
                      'section_number': section_numbers}).set_index('section_number')
    return df

def answers(text,key_begin=['Key','\x0c4'],key_end='\x0c'):
    text = cleanup(text)
    key = re.split('|'.join(key_begin), text)[-1].split(key_end)[0]
    key = re.split('|'.join([' 1. ',' 8. ','15. ','22. ']), key)[1:]
    key = [x.split('SECTION')[0] for x in key]
    
    key = [key[0+n]+key[8+n]+key[4+n]+key[12+n] for n in range(4)]
    key = [re.split('|'.join([str(n)+'\. ' for n in range(100)]),x) for x in key]
    key = [''.join(x).replace(' ','') for x in key]
    
    def answer_filter(s):
        out = ''
        filter_open = True
        for char in s:
            if filter_open:
                if char in ['A','B','C','D','E','*']:
                    out += char
                else:
                    filter_open = False
        return out
        
    key = [answer_filter(x) for x in key]
    return key

def stimuli_questions_LR(section_content):
    ### this function is not working on some edge cases ###
    sq = re.split('|'.join([str(n)+'\. ' for n in range(100)]),section_content)[1:]
    sq = [x.split('(A)')[0] for x in sq]
    sq = [re.split('\. |\? ',x) for x in sq]
    
    def drop_empty(x):
        return [y for y in x if y != '']
    sq = [drop_empty(x) for x in sq]
    
    stimuli = [x[:-1] for x in sq]
    stimuli = ['. '.join(x).strip() for x in stimuli]
    questions = [x[-1].strip() for x in sq]
    return stimuli, questions

def choices_LR(section_content):
    ### this function is not working on some edge cases ###
    choices = re.split('|'.join([str(n)+'\. ' for n in range(100)]),section_content)[1:]
    choices = [x.split('(A)') for x in choices]
    return choices

In [515]:
section_content= sections(text38)['section_content'][1]
choices_LR(section_content)

[['Physician:  In itself, exercise does not cause heart  Questions 3\xe2\x80\x934  attacks; rather, a sudden increase in an exercise regimen can be a cause. When people of any physical condition suddenly increase their amount of exercise, they also increase their risk of heart attack. As a result, there will be an increased risk of heart attack among employees of this company due to the new health program.  The conclusion drawn by the physician follows logically if which one of the following is assumed? ',
  "  Employees will abruptly increase their amount of exercise as a result of the new health program. The exercises involved in the new health program are more strenuous than those in the previous health program.  (B)  (C) The new health program will force employees  of all levels of health to exercise regularly.  (D) The new health program constitutes a sudden  (E)  change in the company's policy. All employees, no matter what their physical condition, will participate in the new he

In [329]:
check_function = sections(text38)
for t in [text59, text60, text61, text65]:
    check_function = check_function.append(sections(t))
check_function

,section_content,section_label
section_number,,
1,". 1. Physician: In itself, exercise does not...",Logical Reasoning
2,. Questions 1–7 A car drives into the center ...,Logical Reasoning
3,. (5) (10) (15) (20) (25) (30) The myth...,Analytical Reasoning
4,. Questions 1–2 Ms. Smith: I am upset that m...,Reading Comprehension
1,. Questions 1–5 A law firm has seven departme...,Analytical Reasoning
2,". 1. On the Caribbean island of Guadeloupe, a...",Logical Reasoning
3,1. New technologies that promise to extend l...,Logical Reasoning
4,. Passage A Recent studies have shown that s...,Reading Comprehension
1,. 1. Jim’s teacher asked him to determine...,Logical Reasoning
